In [2]:
# Load necessary packages
import pandas as pd
import os
import fuzzywuzzy
from fuzzywuzzy import fuzz, process # run 'pip install python-Levenshtein' in the terminal before importing this
import rapidfuzz
from rapidfuzz import process, fuzz

/Users/Louise/anaconda3/envs/Webscraping_environment/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
os.getcwd()

'/Users/Louise/Desktop/Master/M1/S2/Data_viz/Project/code'

In [4]:
# Import datasets
df_catalogue = pd.read_csv("../data/intermediate/Scraped_Catalogue.csv")
df_w_cat = pd.read_csv("../data/intermediate/Full-Products-Data/Products_Categories.csv")
#df_w_without = pd.read_csv("../data/intermediate/Full-Products-Data/Products_Only.csv")

df_walmart = df_w_cat.copy()

In [5]:
df_catalogue.head()

,Unnamed: 0,X,Category,item,item_number,kosher,price,weight,category_clean
0,1,1,ASIAN FOOD,A Taste of Thai Green Curry Paste 1.75 oz.,22668-399,False,$2.45,1.75 oz.,ASIAN FOOD
1,2,2,ASIAN FOOD,A Taste of Thai Pad Thai Sauce 3.25 oz.,22668-408,False,$2.95,3.25 oz.,ASIAN FOOD
2,3,3,ASIAN FOOD,A Taste of Thai Panang Curry Paste 1.75 oz.,22668-409,False,$2.45,1.75 oz.,ASIAN FOOD
3,4,4,ASIAN FOOD,A Taste of Thai Peanut Sauce Mix 3.5 oz.,22668-414,False,$2.95,3.5 oz.,ASIAN FOOD
4,5,5,ASIAN FOOD,A Taste of Thai Red Curry Paste 1.75 oz.,3111-998,False,$2.45,1.75 oz.,ASIAN FOOD


In [6]:
df_walmart.head()

,source_file,name,unit_price,unit_type,clean_unit_price
0,walmart_asian_food.json,Freshness Guaranteed Hot and Ready-to-Eat Chic...,NaN,NaN,NaN
1,walmart_bottled-water.json,TAL Stainless Steel Ranger Water Bottle,NaN,NaN,NaN
2,walmart_bread.json,Freshness Guaranteed French Bakery Bread Loaf,NaN,NaN,NaN
3,walmart_bread.json,Freshness Guaranteed Sourdough Bakery Bread Loaf,NaN,NaN,NaN
4,walmart_candy_sugarfree.json,RUSSELL STOVER Sugar Free Caramel Chocolate Ca...,NaN,NaN,NaN


In [14]:
# Create names datasets
walmart_names = df_walmart['name']
catalogue_names = df_catalogue['item']

# Remove all special characters and numbers from the names to keep only letters in lowercase, in the walmart dataset
import re
def clean_name(name):
    # Remove special characters and numbers using regex
    cleaned_name = re.sub(r'[^A-Za-z\s]', '', name)
    # Convert to lowercase
    cleaned_name = cleaned_name.lower()
    return cleaned_name

df_walmart['name'] = df_walmart['name'].apply(clean_name)

# Keep only unique names in the walmart dataset
df_walmart = df_walmart.drop_duplicates(subset=['name'])


In [15]:
# Remove all "number + oz" patterns from the names, and the special characters, to keep only letters in lowercase, in the catalogue dataset
def clean_catalogue_name(name):
    # Remove "number + oz" patterns using regex
    cleaned_name = re.sub(r'\b\d+\s*oz\b', '', name, flags=re.IGNORECASE)
    # Remove special characters and numbers using regex
    cleaned_name = re.sub(r'[^A-Za-z\s]', '', cleaned_name)
    # Convert to lowercase
    cleaned_name = cleaned_name.lower()
    return cleaned_name

df_catalogue['item'] = df_catalogue['item'].apply(clean_catalogue_name)

# Keep only relevant columns in the catalogue dataset
df_catalogue = df_catalogue[['item','kosher','price', 'weight', 'category_clean']]

In [16]:
df_walmart.head()

,source_file,name,unit_price,unit_type,clean_unit_price
0,walmart_asian_food.json,freshness guaranteed hot and readytoeat chicke...,NaN,NaN,NaN
1,walmart_bottled-water.json,tal stainless steel ranger water bottle,NaN,NaN,NaN
2,walmart_bread.json,freshness guaranteed french bakery bread loaf,NaN,NaN,NaN
3,walmart_bread.json,freshness guaranteed sourdough bakery bread loaf,NaN,NaN,NaN
4,walmart_candy_sugarfree.json,russell stover sugar free caramel chocolate ca...,NaN,NaN,NaN


In [17]:
df_catalogue.head()

,item,kosher,price,weight,category_clean
0,a taste of thai green curry paste,False,$2.45,1.75 oz.,ASIAN FOOD
1,a taste of thai pad thai sauce,False,$2.95,3.25 oz.,ASIAN FOOD
2,a taste of thai panang curry paste,False,$2.45,1.75 oz.,ASIAN FOOD
3,a taste of thai peanut sauce mix,False,$2.95,3.5 oz.,ASIAN FOOD
4,a taste of thai red curry paste,False,$2.45,1.75 oz.,ASIAN FOOD


In [19]:
# Function to get the best matches between two datasets
def get_matches(small_df_names, large_df_names):
    results = {}
    
    # Convert large_df_names to string to avoid issues with NoneType
    large_df_names = [str(n) for n in large_df_names if n is not None]
    
    for name in small_df_names:
        if pd.isna(name) or name == "": continue
        
        match = process.extractOne(
            str(name), 
            large_df_names, 
            scorer=fuzz.token_sort_ratio
        )
        
        if match:
            results[name] = {'walmart_match': match[0], 'score': match[1]}
            
    return results

# Execute the matching function with Walkenhorst as reference
matches_dict = get_matches(df_catalogue['item'].unique(), df_walmart['name'].unique())

# Transform the matches dictionary into a DataFrame
df_matches = pd.DataFrame.from_dict(matches_dict, orient='index').reset_index()
df_matches.columns = ['item', 'walmart_match', 'match_score']

In [49]:
# Transform the matches dictionary into a DataFrame
df_matches = pd.DataFrame.from_dict(matches_dict, orient='index').reset_index()
df_matches.columns = ['item', 'walmart_match', 'match_score']
df_matches = df_matches.sort_values(by = 'match_score', ascending = False)

# Keep only matches with a score of 60 or higher
df_matches = df_matches[df_matches['match_score'] >= 65]
df_matches

,item,walmart_match,match_score
3424,smartfood popcorn white cheddar,smartfood popcorn white cheddar,100.0
713,justins almond butter classic,justins classic almond butter,100.0
659,best foods real mayonnaise,best foods real mayonnaise,100.0
3426,smartfood white cheddar popcorn,smartfood popcorn white cheddar,100.0
699,heinz sweet relish,heinz sweet relish,100.0
...,...,...,...
1734,tasty bite indian vegetable tikka masala,tasty bite organic chickpea tikka masala,65.0
751,melindas creamy style habanero wing sauce,buffalo wild wings mango habanero sauce,65.0
2800,marinela choco roles,martins hoagie rolls,65.0
3074,snack factory pretzel crisps buffalo wing,franks redhot kosher buffalo wing sauce,65.0


In [50]:
# Merge with the catalogue dataset
df_merged = pd.merge(df_catalogue, df_matches, on='item', how='right')
len(df_merged)

1610

In [51]:
# Merge with the walmart dataset
df_walmart = df_walmart.drop_duplicates(subset=['name'])
df_final = pd.merge(df_merged, df_walmart, left_on='walmart_match', right_on='name', how='left')

In [52]:
# Rename and keep only relevant columns in the final dataframe
df_final = df_final.rename(columns={'item': 'Walkenhorst_product', 'price': 'Walkenhorst_price', 'weight': 'Walkenhorst_weight', 'category_clean': 'Walkenhorst_Category', 'source_file': 'Walmart_category', 'name' : 'Walmart_product', 'unit_price': 'Walmart_unit_price', 'unit_type' : 'Walmart_unit_type', 'clean_unit_price' : 'Walmart_clean_unit_price' })
df_final.drop(columns=['walmart_match'], inplace=True)
df_final

,Walkenhorst_product,kosher,Walkenhorst_price,Walkenhorst_weight,Walkenhorst_Category,match_score,Walmart_category,Walmart_product,Walmart_unit_price,Walmart_unit_type,Walmart_clean_unit_price
0,smartfood popcorn white cheddar,False,$6.95,6.75 oz.,SNACKS,100.0,walmart_popcorn.json,smartfood popcorn white cheddar,NaN,price per ounce,0.631667
1,justins almond butter classic,True,$1.95,1.15 oz.,"CONDIMENTS, SAUCES & SPREADS",100.0,walmart_spreads.json,justins classic almond butter,NaN,price per ounce,0.695000
2,best foods real mayonnaise,False,$7.95,11.5 oz.,"CONDIMENTS, SAUCES & SPREADS",100.0,walmart_condiments.json,best foods real mayonnaise,NaN,price per ounce,0.271000
3,smartfood white cheddar popcorn,False,$3.75,2 oz.,SNACKS,100.0,walmart_popcorn.json,smartfood popcorn white cheddar,NaN,price per ounce,0.631667
4,heinz sweet relish,True,$3.75,12.7 oz.,"CONDIMENTS, SAUCES & SPREADS",100.0,walmart_condiments.json,heinz sweet relish,NaN,price per ounce,0.206000
...,...,...,...,...,...,...,...,...,...,...,...
1605,tasty bite indian vegetable tikka masala,True,$4.45,10 oz.,MEALS & SIDES,65.0,walmart_asian_food.json,tasty bite organic chickpea tikka masala,NaN,price per ounce,0.384000
1606,melindas creamy style habanero wing sauce,False,$5.45,12 oz.,"CONDIMENTS, SAUCES & SPREADS",65.0,walmart-hot-sauce.json,buffalo wild wings mango habanero sauce,NaN,price per ounce,0.287000
1607,marinela choco roles,False,$2.95,NA oz.,PASTRIES,65.0,walmart_bread.json,martins hoagie rolls,NaN,price per ounce,0.213000
1608,snack factory pretzel crisps buffalo wing,False,$2.45,3 oz.,SNACKS,65.0,walmart_sauces.json,franks redhot kosher buffalo wing sauce,NaN,price per ounce,0.229000


In [53]:
# Export the final dataframe to a CSV file
df_final.to_csv("../data/processed/Merged_Walkenhorst_Walmart.csv")